# 🚗 NYC Motor Vehicle Collisions Data Analysis
### 📌 Objective:
This project analyzes **motor vehicle collision data in NYC** (2020-2025), cleans missing data, and performs exploratory data analysis (EDA). The final dataset is used to identify trends in accidents, borough-wise distributions, high-risk locations, and severity statistics.

---

## 📂 Step 1: Load & Inspect the Dataset

In [2]:
import pandas as pd

# Load the dataset with low_memory=False to prevent mixed type warning
df = pd.read_csv('Motor_Vehicle_Collisions.csv', low_memory=False)

# Display basic information and first few rows to understand the dataset
print(df.info())
print(df.head())

## 📆 Step 2: Convert CRASH DATE to Date Format

In [9]:
# Convert 'CRASH DATE' to datetime type
df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE'], errors='coerce')

        CRASH DATE CRASH TIME   BOROUGH ZIP CODE   LATITUDE  LONGITUDE  \
0       2021-09-11       2:39       NaN      NaN        NaN        NaN   
1       2022-03-26      11:45       NaN      NaN        NaN        NaN   
2       2023-11-01       1:29  BROOKLYN    11230  40.621790 -73.970024   
3       2022-06-29       6:55       NaN      NaN        NaN        NaN   
4       2022-09-21      13:21       NaN      NaN        NaN        NaN   
...            ...        ...       ...      ...        ...        ...   
2158583 2025-02-25      19:06  BROOKLYN    11236  40.640877 -73.907005   
2158584 2025-02-25       0:30    QUEENS    11001  40.729332 -73.708250   
2158585 2025-02-21      16:16  BROOKLYN    11212  40.654358 -73.919150   
2158586 2025-02-25      19:05  BROOKLYN    11211  40.708904 -73.959250   
2158587 2025-02-25      16:52  BROOKLYN    11225  40.663517 -73.957210   

                        LOCATION           ON STREET NAME CROSS STREET NAME  \
0                            NaN

## 🧹 Step 3: Data Cleaning – Remove Unnecessary Columns

In [ ]:
# Filter the data for years 2020 to 2025
df_filtered = df[(df['CRASH DATE'].dt.year >= 2020) & (df['CRASH DATE'].dt.year <= 2025)]

print(df_filtered)
df_filtered.info()

# Save the filtered dataset to a new CSV file
df_filtered.to_csv('motor_vehicle_collisions_2020_2025.csv', index=False)

## 🔄 Step 4: Keep Only Essential Columns

In [ ]:
# Open filtered dataset
mva=pd.read_csv('motor_vehicle_collisions_2020_2025.csv', low_memory=False)

# Define columns to drop
columns_to_remove = [
    "CRASH TIME", "LOCATION", "CONTRIBUTING FACTOR VEHICLE 1", "CONTRIBUTING FACTOR VEHICLE 2",
    "CONTRIBUTING FACTOR VEHICLE 3", "CONTRIBUTING FACTOR VEHICLE 4", "CONTRIBUTING FACTOR VEHICLE 5",
    "COLLISION_ID", "VEHICLE TYPE CODE 1", "VEHICLE TYPE CODE 2", "VEHICLE TYPE CODE 3",
    "VEHICLE TYPE CODE 4", "VEHICLE TYPE CODE 5"
]

# Remove columns
df_cleaned = mva.drop(columns=columns_to_remove, errors="ignore")

# Ensure columns exist before processing
expected_columns = [
    "CRASH DATE", "BOROUGH", "ZIP CODE", "LATITUDE", "LONGITUDE", "ON STREET NAME", 
    "CROSS STREET NAME", "OFF STREET NAME", "NUMBER OF PERSONS INJURED", "NUMBER OF PERSONS KILLED",
    "NUMBER OF PEDESTRIANS INJURED", "NUMBER OF PEDESTRIANS KILLED", "NUMBER OF CYCLIST INJURED",
    "NUMBER OF CYCLIST KILLED", "NUMBER OF MOTORIST INJURED", "NUMBER OF MOTORIST KILLED"
]

# Keep only the required columns
df_cleaned = mva[expected_columns].copy()

## 🧼 Step 5: Data Type Conversions & Cleaning

In [4]:
# Convert CRASH DATE to datetime
df_cleaned["CRASH DATE"] = pd.to_datetime(df_cleaned["CRASH DATE"], errors="coerce")

# Convert BOROUGH to string
df_cleaned["BOROUGH"] = df_cleaned["BOROUGH"].astype(str)

# Convert 'ZIP CODE' to numeric, turning invalid or non-numeric entries into NaN
df_cleaned["ZIP CODE"] = pd.to_numeric(df_cleaned["ZIP CODE"], errors="coerce")

# Convert to the 'Int64' type, which allows NaN values and removes the decimal part
df_cleaned['ZIP CODE'] = df_cleaned['ZIP CODE'].astype('Int64')  # Convert to Int64 to retain NaN and avoid '.0'


# Convert LATITUDE and LONGITUDE to float
df_cleaned["LATITUDE"] = pd.to_numeric(df_cleaned["LATITUDE"], errors="coerce")
df_cleaned["LONGITUDE"] = pd.to_numeric(df_cleaned["LONGITUDE"], errors="coerce")

# Convert street names to string (handling missing values)
street_columns = ["ON STREET NAME", "CROSS STREET NAME", "OFF STREET NAME"]
df_cleaned[street_columns] = df_cleaned[street_columns].astype(str).replace("<NA>", "")

# Convert injury and fatality columns to integers
injury_columns = [
    "NUMBER OF PERSONS INJURED", "NUMBER OF PERSONS KILLED", "NUMBER OF PEDESTRIANS INJURED",
    "NUMBER OF PEDESTRIANS KILLED", "NUMBER OF CYCLIST INJURED", "NUMBER OF CYCLIST KILLED",
    "NUMBER OF MOTORIST INJURED", "NUMBER OF MOTORIST KILLED"
]
df_cleaned[injury_columns] = df_cleaned[injury_columns].fillna(0).astype(int)

       CRASH DATE   BOROUGH  ZIP CODE   LATITUDE  LONGITUDE  \
0      2021-09-11       nan      <NA>        NaN        NaN   
1      2022-03-26       nan      <NA>        NaN        NaN   
2      2023-11-01  BROOKLYN     11230  40.621790 -73.970024   
3      2022-06-29       nan      <NA>        NaN        NaN   
4      2022-09-21       nan      <NA>        NaN        NaN   
...           ...       ...       ...        ...        ...   
526668 2025-02-25  BROOKLYN     11236  40.640877 -73.907005   
526669 2025-02-25    QUEENS     11001  40.729332 -73.708250   
526670 2025-02-21  BROOKLYN     11212  40.654358 -73.919150   
526671 2025-02-25  BROOKLYN     11211  40.708904 -73.959250   
526672 2025-02-25  BROOKLYN     11225  40.663517 -73.957210   

                 ON STREET NAME CROSS STREET NAME       OFF STREET NAME  \
0         WHITESTONE EXPRESSWAY         20 AVENUE                   nan   
1       QUEENSBORO BRIDGE UPPER               nan                   nan   
2                 

## 💾 Step 6: Save Cleaned Dataset

In [ ]:
# Save cleaned data to a new CSV file
df_cleaned.to_csv("cleaned_motor_vehicle_collisions_2020_2025.csv", index=False)
print(df_cleaned)
df_cleaned.info()

## 🗺️ Step 7: Fill Missing BOROUGH & ZIP CODE Using Geopy

In [ ]:
#Instal GeoPy Library
!pip install geopy pandas tqdm

In [ ]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from tqdm import tqdm
import time

# Load the dataset
df = pd.read_csv('cleaned_motor_vehicle_collisions_2020_2025.csv')

# Initialize geocoder with a user-agent
geolocator = Nominatim(user_agent="geoapi", timeout=10)

# Function to get ZIP code from Latitude and Longitude
def get_zip_code(lat, lon, retries=3):
    try:
        if pd.isna(lat) or pd.isna(lon):
            return None  # Return None if coordinates are missing
        
        lat, lon = float(lat), float(lon)  # Ensure lat/lon are floats
        
        # Try to reverse geocode with retry logic
        for _ in range(retries):
            try:
                location = geolocator.reverse((lat, lon), exactly_one=True)
                if location is None:
                    return None

                address = location.raw.get('address', {})

                # Extract ZIP code
                zip_code = address.get("postcode", None)

                return zip_code  # Only return ZIP CODE
            
            except GeocoderTimedOut:
                print(f"Geocoding timed out for coordinates: {lat}, {lon}. Retrying...")
                time.sleep(2)  # Wait before retrying
            except Exception as e:
                print(f"Error geocoding {lat}, {lon}: {e}")
                return None
        
        return None  # If all retries fail
    
    except ValueError:
        print(f"Invalid coordinates: {lat}, {lon}")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None

# Enable progress bar with tqdm
tqdm.pandas()

# Convert to the 'Int64' type, which allows NaN values and removes the decimal part
df['ZIP CODE'] = df['ZIP CODE'].astype('Int64')

# Filter rows where ZIP CODE is missing before applying geocoding
missing_zip_rows = df['ZIP CODE'].isna()

# Apply geocoding only to missing ZIP CODE rows
df.loc[missing_zip_rows, 'ZIP CODE'] = df.loc[missing_zip_rows].progress_apply(
    lambda row: get_zip_code(row["LATITUDE"], row["LONGITUDE"]), axis=1
)

# Display the updated DataFrame
print(df.head())

# Save the cleaned dataset
df.to_csv('cleaned_data.csv', index=False)

In [8]:
# Step 1: Load the dataset
df = pd.read_csv('cleaned_data.csv')

# Step 2: Load the ZIP Code to Borough mapping dataset
zip_borough_df = pd.read_csv('Zip-Codes-info.csv', header=None, names=['ZIP CODE', 'BOROUGH_MAPPING'])

# Step 3: Ensure ZIP CODE columns are strings and clean format
df['ZIP CODE'] = df['ZIP CODE'].astype(str).str.strip().str.split('.').str[0]  # Remove decimals if present
zip_borough_df['ZIP CODE'] = zip_borough_df['ZIP CODE'].astype(str).str.strip()

# Step 4: Merge dataframes on ZIP CODE
df = df.merge(zip_borough_df, on='ZIP CODE', how='left')

# Step 5: Fill missing BOROUGH values with the mapped data
df['BOROUGH'] = df['BOROUGH'].fillna(df['BOROUGH_MAPPING'])

# Step 6: Drop the extra column after filling
df = df.drop(columns=['BOROUGH_MAPPING'])

# Step 7: Remove Duplicate Rows Before Further Processing
df = df.drop_duplicates()
print(f"✅ Removed duplicate rows. New dataset shape: {df.shape}")

# Step 8: Save or display the updated dataset
df.to_csv('complete_clean_dataset.csv', index=False)
print(df.head())  # Check the updated dataset


✅ Removed duplicate rows. New dataset shape: (523325, 16)
   CRASH DATE   BOROUGH ZIP CODE  LATITUDE  LONGITUDE  \
0  2021-09-11       NaN      nan       NaN        NaN   
1  2022-03-26       NaN      nan       NaN        NaN   
2  2023-11-01  BROOKLYN    11230  40.62179 -73.970024   
3  2022-06-29       NaN      nan       NaN        NaN   
4  2022-09-21       NaN      nan       NaN        NaN   

            ON STREET NAME CROSS STREET NAME OFF STREET NAME  \
0    WHITESTONE EXPRESSWAY         20 AVENUE             NaN   
1  QUEENSBORO BRIDGE UPPER               NaN             NaN   
2            OCEAN PARKWAY          AVENUE K             NaN   
3       THROGS NECK BRIDGE               NaN             NaN   
4          BROOKLYN BRIDGE               NaN             NaN   

   NUMBER OF PERSONS INJURED  NUMBER OF PERSONS KILLED  \
0                          2                         0   
1                          1                         0   
2                          1            

# 📌 Standardizing Street Names in the Dataset

### **🔹 Objective**
This section standardizes street names by converting abbreviations (e.g., `AVE` → `AVENUE`, `ST` → `STREET`) into their full forms to ensure consistency in the dataset. 


In [ ]:
import re

# ✅ Load the dataset
df = pd.read_csv('complete_clean_dataset.csv')

# ✅ Define a function to standardize street names
def standardize_address(address):
    if pd.isna(address):
        return address  # Return NaN values as-is

    # Convert to uppercase for consistency
    address = address.upper().strip()

    # Define standard replacements (only at the end of the string)
    replacements = {
        r'\bAVE\b': 'AVENUE',
        r'\bST\b': 'STREET',
        r'\bBLVD\b': 'BOULEVARD',
        r'\bRD\b': 'ROAD',
        r'\bPL\b': 'PLACE',
        r'\bPKWY\b': 'PARKWAY',
        r'\bDR\b': 'DRIVE'
    }

    # Apply regex replacements only when abbreviation is at the end
    for pattern, replacement in replacements.items():
        address = re.sub(pattern + r'$', replacement, address)

    return address

# ✅ Apply function to the ON STREET NAME column
df["Standardized Address ON STREET NAME"] = df["ON STREET NAME"].apply(standardize_address)
# ✅ Apply function to the CROSS STREET NAME column
df["Standardized Address CROSS STREET NAME"] = df["CROSS STREET NAME"].apply(standardize_address)
# ✅ Apply function to the OFF STREET NAME column
df["Standardized Address OFF STREET NAME"] = df["OFF STREET NAME"].apply(standardize_address)

# ✅ Save cleaned dataset for Tableau
df.to_csv("cleaned_dataset_with_standard_addresses.csv", index=False)

# ✅ Print sample output to verify
print(df[["ON STREET NAME", "Standardized Address ON STREET NAME"]].head(20))

## 📊 Step 8: Exploratory Data Analysis (EDA)

In [28]:
# Step 1: Load the main dataset
df_dataset = pd.read_csv('cleaned_dataset_with_standard_addresses.csv')

# Summary statistics on accident counts per year
df_dataset["CRASH DATE"] = pd.to_datetime(df_dataset["CRASH DATE"], errors="coerce")
df_dataset["YEAR"] = df_dataset["CRASH DATE"].dt.year
yearly_accidents = df_dataset["YEAR"].value_counts().sort_index()

# Borough-wise accident distribution
borough_accidents = df_dataset["BOROUGH"].value_counts()

# Most dangerous locations (streets with the highest number of accidents)
top_streets = df_dataset["Standardized Address ON STREET NAME"].value_counts().dropna().head(10)

# Total injuries and fatalities
total_injuries = df_dataset["NUMBER OF PERSONS INJURED"].sum()
total_fatalities = df_dataset["NUMBER OF PERSONS KILLED"].sum()

# Injury distribution among different groups
pedestrian_injuries = df_dataset["NUMBER OF PEDESTRIANS INJURED"].sum()
cyclist_injuries = df_dataset["NUMBER OF CYCLIST INJURED"].sum()
motorist_injuries = df_dataset["NUMBER OF MOTORIST INJURED"].sum()

# Fatality distribution among different groups
pedestrian_fatalities = df_dataset["NUMBER OF PEDESTRIANS KILLED"].sum()
cyclist_fatalities = df_dataset["NUMBER OF CYCLIST KILLED"].sum()
motorist_fatalities = df_dataset["NUMBER OF MOTORIST KILLED"].sum()

# Display results
(yearly_accidents, borough_accidents, top_streets, 
 total_injuries, total_fatalities, 
 pedestrian_injuries, cyclist_injuries, motorist_injuries,
 pedestrian_fatalities, cyclist_fatalities, motorist_fatalities)

(YEAR
 2020    111940
 2021    109872
 2022    103193
 2023     96091
 2024     90826
 2025     11403
 Name: count, dtype: int64,
 BOROUGH
 BROOKLYN         159328
 QUEENS           135172
 BRONX             83568
 MANHATTAN         79882
 STATEN ISLAND     23958
 Name: count, dtype: int64,
 Standardized Address ON STREET NAME
 BELT PARKWAY                  7924
 LONG ISLAND EXPRESSWAY        4810
 BROOKLYN QUEENS EXPRESSWAY    4740
 GRAND CENTRAL PARKWAY         4707
 BROADWAY                      4358
 FDR DRIVE                     4095
 ATLANTIC AVENUE               3748
 MAJOR DEEGAN EXPRESSWAY       3340
 CROSS ISLAND PARKWAY          3272
 3 AVENUE                      2772
 Name: count, dtype: int64,
 np.int64(262104),
 np.int64(1430),
 np.int64(43235),
 np.int64(26221),
 np.int64(183749),
 np.int64(613),
 np.int64(124),
 np.int64(634))

## What This Shows
### 📉 Accident trends (monthly, yearly)
### 📍 Boroughs with the most accidents
### 🚑 Total injuries & fatalities per year
### 🚦 Most dangerous streets in NYC

In [27]:
# Convert CRASH DATE to datetime format
df_dataset["CRASH DATE"] = pd.to_datetime(df_dataset["CRASH DATE"], errors="coerce")

# Extract year and month
df_dataset["YEAR"] = df_dataset["CRASH DATE"].dt.year
df_dataset["MONTH"] = df_dataset["CRASH DATE"].dt.month

# Ensure df_new includes these columns
df_dataset = df_dataset.copy()  # Create a fresh copy with the correct columns

# 1. Accident trends over time (monthly and yearly)
monthly_accidents = df_dataset.groupby(["YEAR", "MONTH"]).size().unstack()
print("Monthly Accident Trends:\n", monthly_accidents)

# 2. Accident distribution by borough
borough_accidents = df_dataset["BOROUGH"].value_counts()
print("\nAccident Distribution by Borough:\n", borough_accidents)

# 3. Severity analysis: Injuries & Fatalities per year
injuries_per_year = df_dataset.groupby("YEAR")["NUMBER OF PERSONS INJURED"].sum()
fatalities_per_year = df_dataset.groupby("YEAR")["NUMBER OF PERSONS KILLED"].sum()
print("\nInjuries per Year:\n", injuries_per_year)
print("\nFatalities per Year:\n", fatalities_per_year)

# 4. High-risk locations: Top 10 streets with most accidents
top_streets = df_dataset["Standardized Address ON STREET NAME"].value_counts().head(10)
print("\nTop 10 Most Dangerous Streets:\n", top_streets)

Monthly Accident Trends:
 MONTH       1        2        3       4        5        6       7       8   \
YEAR                                                                         
2020   14191.0  13550.0  10958.0  4095.0   6125.0   7587.0  9222.0  9743.0   
2021    7674.0   6939.0   8205.0  8714.0  10224.0  10548.0  9936.0  9816.0   
2022    7853.0   7342.0   8815.0  8486.0   9391.0   9420.0  8832.0  8868.0   
2023    7429.0   6855.0   8140.0  7730.0   8940.0   8142.0  8192.0  7967.0   
2024    7508.0   6803.0   7818.0  7124.0   8153.0   7932.0  7452.0  7590.0   
2025    6394.0   5009.0      NaN     NaN      NaN      NaN     NaN     NaN   

MONTH      9        10      11      12  
YEAR                                    
2020   9531.0   9640.0  8971.0  8327.0  
2021   9831.0  10153.0  9311.0  8521.0  
2022   8828.0   8894.0  8129.0  8335.0  
2023   8251.0   8682.0  7815.0  7948.0  
2024   7796.0   7766.0  7320.0  7564.0  
2025      NaN      NaN     NaN     NaN  

Accident Distributio